In [1]:
import cv2
from my_function import generate_gaussian_noise

# 视频处理

视频处理的核心就是对视频帧图像进行处理

### 视频读取与播放

OpenCV提供了`cv2.VideoCapture`和`cv2.VideoWriter`两个类，分别用于视频的读取和写入

`cv2.VideoCapture(0)`表示从摄像头读取视频

In [ ]:
# 创建 VideoCapture 对象，读取视频文件
cap=cv2.VideoCapture("./images/oceans.mp4")
# 检查视频是否成功打开
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()
# 读取视频帧
while True:
    ret, frame = cap.read() # ret为布尔值，表示是否成功读取帧
    # 如果读取到最后一帧，退出循环
    if not ret:
        break
    # 显示当前帧
    cv2.imshow('Video', frame)
    # 按下 'q' 键退出
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
# 释放资源
cap.release()
cv2.destroyAllWindows()

`0xFF` 在 OpenCV 的按键处理中是一个掩码（mask），用于从 `cv2.waitKey()` 的返回值中提取正确的按键信息

### 视频帧处理

在读取视频帧后，可以对每一帧进行各种图像处理操作

下面将视频帧转换为灰度图并做高斯噪声处理，然后进行双边滤波处理

在对每个帧处理时，建议使用dst参数以减小重复使用内存分配带来的性能开销

In [4]:
cap=cv2.VideoCapture("./images/oceans.mp4")
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()
while True:
    ret, frame = cap.read()
    if not ret:
        break
    new_frame=cv2.bilateralFilter(
        generate_gaussian_noise(
            cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY),std=25),9,75,75)
    cv2.imshow('gray Video',new_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

### 视频帧保存

使用`cv2.VideoWriter`类来实现视频保存

视频的几个属性：
- 帧率：每秒显示的帧数：`CAP_PROP_FPS`
- 尺寸：视频帧的宽度和高度：`CAP_PROP_FRAME_WIDTH`，`CAP_PROP_FRAME_HEIGHT`
- 编码格式：使用`cv2.VideoWriter_fourcc`函数指定编码格式，编码格式有很多种，如`XVID`、`MJPG`、`H264`等

需要注意的是：cv2.VideoWriter生成的视频编码格式兼容性差，默认使用 BGR颜色顺序，导致生成的视频在vscode中无法播放，需要进行颜色顺序的转换：

`frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)`

对灰度图，需要转换为三通道图像：

`frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2RGB)`

VSCode支持的预览视频格式有：`H.264`，`VP8`，要想在VSCode中预览视频，建议使用`H.264`编码格式，并保存为`.mp4`格式文件

In [6]:
cap=cv2.VideoCapture("./images/oceans.mp4")
# 获取视频的帧率和尺寸
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
# 创建 VideoWriter 对象，保存处理后的视频
fourcc = cv2.VideoWriter_fourcc(*'H264')
out = cv2.VideoWriter('./images/gray_oceans.mp4', fourcc, fps, (width, height))
while True:
    ret, frame = cap.read()
    if not ret:
        break
    new_frame=cv2.bilateralFilter(
        generate_gaussian_noise(
            cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY),std=25),9,75,75)
    # 将帧写入输出视频
    out.write(cv2.cvtColor(new_frame, cv2.COLOR_GRAY2RGB))
    # 显示帧
    cv2.imshow('Gray Video', new_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()